In [1]:
import argparse
import json
import logging
import os
import random
import time

import torch
import numpy as np


In [2]:
import sys 
sys.path.append('..')
import utils.misc as ws
import utils.data_utils
import utils.train_utils
import utils.eval_utils
import utils.mesh
import utils.dataset as d
import models.networks as arch


In [3]:
DATA_SOURCE = '/home/ninad/Desktop/Docs/phd-res/proj-irvl-grasp-transfer/code/docker-data/output_dataset/'
EXPERIMENTS_DIR = '../experiments/initial_experiment/'
CHECKPOINT = 'latest'
split_filename = '../split_validation.json'
specs_filename = os.path.join(EXPERIMENTS_DIR, "specs.json")

In [4]:
specs = json.load(open(specs_filename))
latent_size = specs["CodeLength"]
gripper_weight = specs["GripperWeight"]

In [5]:
decoder = arch.dsdfDecoder(latent_size, **specs["NetworkSpecs"])

decoder = torch.nn.DataParallel(decoder)

saved_model_state = torch.load(
    os.path.join(
        EXPERIMENTS_DIR, ws.model_params_subdir, CHECKPOINT + ".pth")
)

saved_model_epoch = saved_model_state["epoch"]

decoder.load_state_dict(saved_model_state["model_state_dict"])

decoder = decoder.module.cuda()

In [6]:
with open(split_filename, "r") as f:
    split = json.load(f)

# npz_filenames = utils.data_utils.dsdf_get_instance_filenames(
#     args.data_source, split)
npz_filenames = utils.data_utils.get_instance_filelist(DATA_SOURCE, split)

# random.shuffle(npz_filenames) # WHY???

In [7]:
reconstruction_dir = os.path.join(
    EXPERIMENTS_DIR, ws.reconstructions_subdir, str(saved_model_epoch))

reconstruction_codes_dir = os.path.join(
    reconstruction_dir,
    ws.reconstruction_codes_subdir)

In [8]:
index_to_select = random.randint(0, len(npz_filenames)-1)

npz = npz_filenames[index_to_select]

full_filename = npz

print(index_to_select)

0


In [9]:
logging.debug("loading {}".format(npz))

# data_sdf = utils.data_utils.read_sdf_samples_into_ram(full_filename)

filename_split_list = os.path.normpath(full_filename).split(os.path.sep)

npz_file_info = '-'.join(filename_split_list[-4:])[:-4]

latent_filename = os.path.join(reconstruction_codes_dir, npz_file_info + ".pth")

latent_data = torch.load(latent_filename)

In [10]:
latent_data.shape

torch.Size([1, 1, 256])

In [11]:
type(latent_data)
latent_data.squeeze().shape

torch.Size([256])

In [12]:
sdf_dataset = d.SDFSamples(DATA_SOURCE, split, 1000000)

sdf_loader = torch.utils.data.DataLoader(
    sdf_dataset,
    batch_size=1,
    shuffle=True,
    num_workers=8,
    drop_last=True )

In [ ]:
# samples, npzfile = next(iter(sdf_loader))
samples, idx, npzfile = sdf_dataset[index_to_select]
print(npzfile[-40:])

queries = samples[:, :3] # Need to pass this through the network
gt_sdf_obj = samples[:, 3].squeeze().numpy()
gt_sdf_grp = samples[:, 4].squeeze().numpy()
print(gt_sdf_grp.shape)

In [ ]:
samples.shape

In [ ]:
# latent_data = latent_data.squeeze(1)
# with torch.no_grad():
#     queries, sdf_obj, sdf_grp = utils.eval_utils.eval_query_pc(decoder, latent_data, samples.squeeze())

In [ ]:
print(latent_data.shape)

In [ ]:
# with torch.no_grad():
#     queries, sdf_obj, sdf_grp = utils.eval_utils.eval_query_pc(decoder, latent_data.squeeze())

In [13]:
mesh_filename = 'test.ply'
latent_data = latent_data.squeeze()
with torch.no_grad():
    utils.mesh.create_mesh_custom(
        decoder, latent_data.squeeze(), mesh_filename, N=256, max_batch=int(2 ** 18))

# with torch.no_grad():
#     queries, sdf_obj, sdf_grp = utils.eval_utils.eval_random_query_pc(decoder, latent_data.squeeze())

sampling takes: 11.461635


In [ ]:
queries = queries.detach().cpu().numpy()
sdf_obj = sdf_obj.detach().cpu().numpy()
sdf_grp = sdf_grp.detach().cpu().numpy()

In [ ]:
sdf_obj.shape

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

def plot_sdf(xyz, sdf, title='Sample_Title', n_display=10000):
    fig = plt.figure(figsize = (10,10))
    ax = fig.add_subplot(111, projection='3d')    

    ind = np.random.choice(range(xyz.shape[0]), n_display)
    data = xyz[ind].T

    ax.scatter(data[0], data[2], data[1], s=5, c=sdf[ind])
    ax.view_init(20, 100)
    limit = (-0.95, 0.95)
    ax.set_xlim3d(*limit)
    ax.set_ylim3d(*limit)
    ax.set_zlim3d(*limit)
    plt.title(title)
    plt.show()

In [ ]:
EPS = -1e-4
# ind_obj =  np.where((y >= -EPS) & (y <= EPS))
ind_grp = sdf_grp <= EPS

plot_sdf(queries[ind_grp], sdf_obj[ind_grp])

min(sdf_obj)

In [ ]:
import pyrender
# Just pass the points you want to visualize
def plt_points_3d(pts):
    colors = np.zeros(pts.shape)
    cloud = pyrender.Mesh.from_points(pts, colors=colors)
    scene = pyrender.Scene()
    scene.add(cloud)
    viewer = pyrender.Viewer(scene, use_raymond_lighting=True, point_size=2)

# Pass the point as well as the sdf to see inside/outside points
def plt_points_sdf(pts, sdf, eps=1e-4):
    colors = np.zeros(pts.shape)
    colors[sdf < eps, 1] = 1
    colors[sdf > eps, 0] = 1
    cloud = pyrender.Mesh.from_points(pts, colors=colors)
    scene = pyrender.Scene()
    scene.add(cloud)
    viewer = pyrender.Viewer(scene, use_raymond_lighting=True, point_size=2)

def plt_points_sdf_compare(pts, sdf_gt, sdf_pred):
    colors = np.zeros(pts.shape)
    colors[sdf_gt < 0, 1] = 1
    colors[sdf_gt > 0, 0] = 1
    cloud = pyrender.Mesh.from_points(pts, colors=colors)
    scene = pyrender.Scene()
    scene.add(cloud)
    viewer = pyrender.Viewer(scene, use_raymond_lighting=True, point_size=2)
    
    colors2 = np.zeros(pts.shape)
    colors2[sdf_pred < 0, 1] = 1
    colors2[sdf_pred > 0, 0] = 1
    cloud2 = pyrender.Mesh.from_points(pts, colors=colors2)
    scene2 = pyrender.Scene()
    scene2.add(cloud2)
    viewer = pyrender.Viewer(scene2, use_raymond_lighting=True, point_size=2)

In [ ]:
plt_points_3d(queries[sdf_grp < 1e-4])

In [ ]:
plt_points_sdf(queries, sdf_grp)